In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score, confusion_matrix
from glob import glob
from tqdm import tqdm
import cv2

# --- 설정 ---
SLICE_ROOT = "/data1/lidc-idri/slices"
BATCH_SIZE = 16
NUM_EPOCHS = 100
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- 라벨 추출 ---
def extract_label_from_filename(filename):
    try:
        score = int(filename.split("_")[-1].replace(".npy", ""))
        if score == 3:
            return None
        return 1 if score >= 4 else 0
    except:
        return None

# --- 파일 로딩 및 split ---
all_files = glob(os.path.join(SLICE_ROOT, "LIDC-IDRI-*", "*.npy"))
file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
files, labels = zip(*file_label_pairs)

train_files, temp_files, train_labels, temp_labels = train_test_split(files, labels, test_size=0.3, random_state=42)
val_files, test_files, val_labels, test_labels = train_test_split(temp_files, temp_labels, test_size=0.5, random_state=42)

# --- Dataset ---
class LIDC3ChannelDataset(Dataset):
    def __init__(self, file_paths, labels):
        self.file_paths = file_paths
        self.labels = labels

    def __getitem__(self, idx):
        center_path = self.file_paths[idx]
        label = self.labels[idx]
        folder = os.path.dirname(center_path)
        fname = os.path.basename(center_path)
        slice_num = int(fname.split("_")[1])
        suffix = fname.split("_")[-1]

        indices = [slice_num - 1, slice_num, slice_num + 1]
        imgs = []
        for sn in indices:
            path = os.path.join(folder, f"slice_{sn:03d}_{suffix}")
            if os.path.exists(path):
                img = np.load(path).astype(np.float32)
            else:
                img = np.load(center_path).astype(np.float32)
            img = np.clip(img, -1000, 400)
            img = (img + 1000) / 1400.0
            img = cv2.resize(img, (224, 224))
            imgs.append(img)
        img = np.stack(imgs, axis=0)
        return torch.tensor(img).float(), torch.tensor(label).float()

    def __len__(self):
        return len(self.file_paths)

# --- DataLoader ---
train_loader = DataLoader(LIDC3ChannelDataset(train_files, train_labels), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(LIDC3ChannelDataset(val_files, val_labels), batch_size=BATCH_SIZE)
test_loader = DataLoader(LIDC3ChannelDataset(test_files, test_labels), batch_size=BATCH_SIZE)

# --- 모델 정의 ---
import torchvision.models as models
model = models.resnet18(pretrained=True)
model.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = nn.Linear(model.fc.in_features, 1)
model = model.to(DEVICE)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

save_dir = os.path.join(os.path.dirname(os.getcwd()), "pth")
os.makedirs(save_dir, exist_ok=True)
best_val_acc = 0.0

# --- 학습 루프 ---
for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}"):
        imgs, labels = imgs.to(DEVICE), labels.unsqueeze(1).to(DEVICE)
        optimizer.zero_grad()
        outputsput = model(imgs)
        loss = criterion(outputsput, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"[Epoch {epoch+1}] Train Loss: {running_loss / len(train_loader):.4f}")

    # 검증
    model.eval()
    correct = total = 0
    y_pred, y_true = [], []
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            outputs = model(imgs).squeeze()
            probs = torch.sigmoid(outputs)
            preds = (probs > 0.5).long()
            correct += (preds == labels.long()).sum().item()
            total += labels.size(0)
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
    val_acc = correct / total
    print(f"Validation Accuracy: {val_acc:.4f}")
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), os.path.join(save_dir, "best_model_resnet18_3ch.pth"))
        print("✅ Best model saved!")

# --- 테스트 ---
print("\n\U0001F4CA Test Set Evaluation:")
model.load_state_dict(torch.load(os.path.join(save_dir, "best_model_resnet18_3ch.pth")))
model.eval()
y_pred, y_true, y_probs = [], [], []

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        outputs = model(imgs).squeeze()
        probs = torch.sigmoid(outputs)
        preds = (probs > 0.5).long()
        y_probs.extend(probs.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

print(f"\u2705 Test Accuracy: {(np.array(y_pred) == np.array(y_true)).mean() * 100:.2f}%")
print(classification_report(y_true, y_pred, digits=4))

try:
    auc_score = roc_auc_score(y_true, y_probs)
    print(f"AUC: {auc_score:.4f}*100")
except ValueError:
    print("AUC \uacc4\uc0b0 \uc2e4\ud328: \uc591/\uc74c \ud074\ub798\uc2a4\uac00 \ubaa8\ub450 \uc788\uc5b4\uc57c \ud568.")

cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)



/opt/anaconda3/envs/lungcancer/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/lungcancer/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/100: 100%|██████████| 234/234 [00:14<00:00, 16.71it/s]


[Epoch 1] Train Loss: 0.5673
Validation Accuracy: 0.6987
✅ Best model saved!


Epoch 2/100: 100%|██████████| 234/234 [00:12<00:00, 18.04it/s]


[Epoch 2] Train Loss: 0.3901
Validation Accuracy: 0.7950
✅ Best model saved!


Epoch 3/100: 100%|██████████| 234/234 [00:13<00:00, 17.67it/s]


...

[Epoch 98] Train Loss: 0.0148
Validation Accuracy: 0.9012


Epoch 99/100: 100%|██████████| 234/234 [00:15<00:00, 14.75it/s]


[Epoch 99] Train Loss: 0.0037
Validation Accuracy: 0.9075


Epoch 100/100: 100%|██████████| 234/234 [00:13<00:00, 17.87it/s]


[Epoch 100] Train Loss: 0.0026
Validation Accuracy: 0.9125

📊 Test Set Evaluation:
✅ Test Accuracy: 88.88%
              precision    recall  f1-score   support

         0.0     0.9043    0.7564    0.8238       275
         1.0     0.8825    0.9581    0.9187       525

    accuracy                         0.8888       800
   macro avg     0.8934    0.8572    0.8712       800
weighted avg     0.8900    0.8888    0.8861       800

AUC: 0.9425
Confusion Matrix:
[[208  67]
 [ 22 503]]
